# Параллельные вычисления (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Параллельные вычисления"
* https://nalepae.github.io/pandarallel/
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_windows.ipynb
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_mac_linux.ipynb
* https://requests.readthedocs.io/en/latest/
* https://docs.python.org/3/library/pathlib.html
* https://realpython.com/python-pathlib/
* https://realpython.com/python-gil/
* https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.ThreadPool

## Задачи для совместного разбора

1. Выведите на экран слова из файла words_alpha, в которых есть две или более буквы "e" подряд.

In [1]:
!pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16665 sha256=ff43cfcc4e9272c6f76c9145be666f24603d6beabe930e8b2fb45b9043a28c17
  Stored in directory: c:\users\max sharapov\appdata\local\pip\cache\wheels\62\1e\e7\f9ee096e5cc02890a6934a5670ff6e45a3400f330605bd8210
Successfully built pandarallel


2. Загрузите данные о комментариях с сайта jsonplaceholder.typicode.com

![](https://i.imgur.com/AwiN8y6.png)

In [156]:
import multiprocessing
import numpy as np

3. Получите множество уникальных почтовых доменов.

![](https://i.imgur.com/ceY6guU.png)

## Лабораторная работа 6

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

<p class="task" id="1"></p>

1\. Напишите функцию `f`, которая принимает на вход тэг и проверяет, удовлетворяет ли тэг следующему шаблону: `[любое число]-[любое слово]-or-less`. Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи метода _серий_ `map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

In [157]:
import pandas as pd
import time
import re

def f(tag):
    pattern = r'^\d+-\w+-or-less$'
    if re.match(pattern, tag): ## проверяет, соответствует ли заданный шаблон match ищет совпадения только в начале строки
        return True
    else:
        return False

In [158]:
%%time
df = pd.read_csv('tag_nsteps_10m.csv')
df['tags'] = df['tags'].astype(str) #Нужно чтобы можно было выполнить регулярное выражение на каждой строке
count = df['tags'].map(f).sum()

print(count)

288503
Wall time: 18.5 s


TypeError: expected string or bytes-like object

<!-- TODO -->

<p class="task" id="2"></p>

2\. Напишите функцию `parallel_map`, которая принимает на вход серию `s` `pd.Series` и функцию одного аргумента `f` и поэлементно применяет эту функцию к серии, распараллелив вычисления при помощи пакета `multiprocessing`. Логика работы функции `parallel_map` должна включать следующие действия:
* разбиение исходной серии на $K$ частей, где $K$ - количество ядер вашего процессора;
* параллельное применение функции `f` к каждой части при помощи метода _серии_ `map` c использованием нескольких подпроцессов;
* объединение результатов работы подпроцессов в одну серию. 

Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи `parallel_map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

**Оно невозможное**

<p class="task" id="3"></p>

3\. Используя пакет `pandarallel`, примените функцию `f` из задания 1 к столбцу `tags` таблицы, с которой вы работали в этом задании. Посчитайте количество тэгов, подходящих под описанный шаблон. Выведите на экран полученный результат. Измерьте время выполнения кода. 

In [159]:
import pandas as pd
import time
from pandarallel import pandarallel
import re

pandarallel.initialize()

def f(tag):
    import re
    pattern = r'^\d+-\w+-or-less$'
    if re.match(pattern, tag):
        return True
    else:
        return False

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [160]:
%%time
df = pd.read_csv('tag_nsteps_10m.csv')
df['tags'] = df['tags'].astype(str) 

# применение функции f к столбцу tags с помощью pandarallel
count = df['tags'].parallel_apply(f).sum()

print(count)

288503
Wall time: 15.5 s


В отличие от .map(), который применяет функцию последовательно, .parallel_apply() применяет функцию параллельно на нескольких ядрах, что может ускорить выполнение кода для больших данных.

<p class="task" id="4"></p>

4\. Сайт [DummyJSON](https://dummyjson.com/) позволяет получить набор данных о товарах в виде JSON. Воспользовавшись пакетом `requests`, получите данные о __50 товарах__ и создайте словарь, где ключом является название товара (title), а значением - список ссылок на изображения этого товара. При создании словаря замените символ `/` в названии на пробел.

Выведите на экран количество элементов полученного словаря.

In [161]:
import requests
import json
from tqdm import tqdm

In [162]:
url = requests.get('https://dummyjson.com/products?limit=50')
comments = url.json()
a=comments['products']
pd.DataFrame(a)

,id,title,description,price,discountPercentage,rating,stock,brand,category,thumbnail,images
0,1,iPhone 9,An apple mobile which is nothing like apple,549,12.96,4.69,94,Apple,smartphones,https://i.dummyjson.com/data/products/1/thumbn...,[https://i.dummyjson.com/data/products/1/1.jpg...
1,2,iPhone X,"SIM-Free, Model A19211 6.5-inch Super Retina H...",899,17.94,4.44,34,Apple,smartphones,https://i.dummyjson.com/data/products/2/thumbn...,[https://i.dummyjson.com/data/products/2/1.jpg...
2,3,Samsung Universe 9,Samsung's new variant which goes beyond Galaxy...,1249,15.46,4.09,36,Samsung,smartphones,https://i.dummyjson.com/data/products/3/thumbn...,[https://i.dummyjson.com/data/products/3/1.jpg]
3,4,OPPOF19,OPPO F19 is officially announced on April 2021.,280,17.91,4.30,123,OPPO,smartphones,https://i.dummyjson.com/data/products/4/thumbn...,[https://i.dummyjson.com/data/products/4/1.jpg...
4,5,Huawei P30,Huawei’s re-badged P30 Pro New Edition was off...,499,10.58,4.09,32,Huawei,smartphones,https://i.dummyjson.com/data/products/5/thumbn...,[https://i.dummyjson.com/data/products/5/1.jpg...
5,6,MacBook Pro,MacBook Pro 2021 with mini-LED display may lau...,1749,11.02,4.57,83,Apple,laptops,https://i.dummyjson.com/data/products/6/thumbn...,[https://i.dummyjson.com/data/products/6/1.png...
6,7,Samsung Galaxy Book,Samsung Galaxy Book S (2020) Laptop With Intel...,1499,4.15,4.25,50,Samsung,laptops,https://i.dummyjson.com/data/products/7/thumbn...,[https://i.dummyjson.com/data/products/7/1.jpg...
7,8,Microsoft Surface Laptop 4,Style and speed. Stand out on HD video calls b...,1499,10.23,4.43,68,Microsoft Surface,laptops,https://i.dummyjson.com/data/products/8/thumbn...,[https://i.dummyjson.com/data/products/8/1.jpg...
8,9,Infinix INBOOK,Infinix Inbook X1 Ci3 10th 8GB 256GB 14 Win10 ...,1099,11.83,4.54,96,Infinix,laptops,https://i.dummyjson.com/data/products/9/thumbn...,[https://i.dummyjson.com/data/products/9/1.jpg...
9,10,HP Pavilion 15-DK1056WM,HP Pavilion 15-DK1056WM Gaming Laptop 10th Gen...,1099,6.18,4.43,89,HP Pavilion,laptops,https://i.dummyjson.com/data/products/10/thumb...,[https://i.dummyjson.com/data/products/10/1.jp...


In [169]:
products = {}

for product in a:
    title= product["title"].replace('/',' ')
    products[title] = product['images']
products

{'iPhone 9': ['https://i.dummyjson.com/data/products/1/1.jpg',
  'https://i.dummyjson.com/data/products/1/2.jpg',
  'https://i.dummyjson.com/data/products/1/3.jpg',
  'https://i.dummyjson.com/data/products/1/4.jpg',
  'https://i.dummyjson.com/data/products/1/thumbnail.jpg'],
 'iPhone X': ['https://i.dummyjson.com/data/products/2/1.jpg',
  'https://i.dummyjson.com/data/products/2/2.jpg',
  'https://i.dummyjson.com/data/products/2/3.jpg',
  'https://i.dummyjson.com/data/products/2/thumbnail.jpg'],
 'Samsung Universe 9': ['https://i.dummyjson.com/data/products/3/1.jpg'],
 'OPPOF19': ['https://i.dummyjson.com/data/products/4/1.jpg',
  'https://i.dummyjson.com/data/products/4/2.jpg',
  'https://i.dummyjson.com/data/products/4/3.jpg',
  'https://i.dummyjson.com/data/products/4/4.jpg',
  'https://i.dummyjson.com/data/products/4/thumbnail.jpg'],
 'Huawei P30': ['https://i.dummyjson.com/data/products/5/1.jpg',
  'https://i.dummyjson.com/data/products/5/2.jpg',
  'https://i.dummyjson.com/data/pr

In [35]:
len(products)

50

<p class="task" id="5"></p>

5\. Напишите функцию `download_product_imgs`, которая создает папку c названием товара внутри каталога `imgs` (сам каталог `imgs` может быть создан любым удобным способом до начала работы) и сохраняет в нее изображения. Название для файла изображения извлеките из URL.

Воспользовавшись этой функцией, скачайте изображения всех продуктов. Выведите на экран общее количество загруженных файлов. Для отслеживания хода выполнения кода используйте пакет `tqdm`.

In [55]:
from tqdm import tqdm

In [56]:
from pathlib import Path
Path('imgs').mkdir(exist_ok=True)

In [57]:
# пример кода для скачивания картинки
url = "https://png.pngtree.com/png-vector/20201229/ourmid/pngtree-a-british-short-blue-and-white-cat-png-image_2654518.jpg"
img = requests.get(url).content
with open("cat.jpg", "wb") as fp:
    fp.write(img)

In [58]:
def download_product_imgs(title, imgs):
    root = Path(f'./imgs/{title}')
    root.mkdir(exist_ok=True)
    for url in imgs:
        img = requests.get(url).content
        filename = os.path.basename(url) # возвращает имя файла
        with open(root / f"{filename}.jpg", "wb") as fp:
            fp.write(img)
    '''
    title - название товара
    imgs - список ссылок на изображения товара
    '''
    return 

In [59]:
for key, value in tqdm(products.items()):
    download_product_imgs(key,value)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:56<00:00,  3.54s/it]


In [60]:
import os
total_files = sum([len(files) for _, _, files in os.walk('imgs')]) #проходит через каждый каталог, файлы которого находятся в папке imgs
total_files 

223

используется для указания на то, что значение переменной не будет использоваться в теле цикла

<p class="task" id="6"></p>

6\. Создайте функцию `download_product_imgs_processes` на основе функции `download_product_imgs`, добавив в нее вывод сообщения следующего вида: `Process ID: <ID текущего процесса>`. Для определения ID процесса воспользуйтесь функцией `multiprocessing.current_process()`.

Решите задачу 5, распараллелив вычисления при помощи процессов. Вместо корневого каталога `imgs` используйте `imgs_processes`. Выведите на экран общее количество загруженных файлов. Измерьте время выполнения кода. 

<p class="task" id="7"></p>

7\. Создайте функцию `download_product_imgs_threads` на основе функции `download_product_imgs`, добавив в нее вывод сообщения следующего вида: `Process ID: <ID текущего процесса> Thread ID: <ID текущего потока>`. Для определения ID потока воспользуйтесь функцией `threading.get_ident`.

Решите задачу 5, распараллелив вычисления при помощи потоков. Вместо корневого каталога `imgs` используйте `imgs_threads`. Выведите на экран общее количество загруженных файлов. Измерьте время выполнения кода. 

<p class="task" id="8"></p>

8\. Напишите функцию `create_2d_list`, которая создает матрицу размера `m` на `n` (__в виде списка списков__) вещественных чисел из стандартного нормального распределения. Напишите функцию `sum_by_chunk`, которая принимает на вход несколько строк этой матрицы (тоже в виде списка списков) и находит сумму элементов. 

Используя данную функцию, решите задачу поиска суммы по всей матрице тремя способами:
* передав в функцию `sum_by_chunk` всю матрицу целиком;
* распараллелив вычисления при помощи процессов по следующему принципу: матрица разбивается на части (например, по 1тыс. строк); процессы независимо друг от друга обрабатывают эти части; после завершения работы всех процессов результаты агрегируются для получения результата для всей матрицы;
* распараллелив вычисления при помощи потоков аналогичным способом.

Для демонстрации результата создайте матрицу достаточно большого размера (не менее 100 тыс. строк), выведите на экран результаты работы трех вариантов решения и измерьте время выполнения каждого из них.

В данном задании разрешается использовать пакет `numpy` только для создания матрицы. В этом случае необходимо преобразовать ее к списку списков до начала работы.

In [70]:
def create_2d_list(m, n):
    matrix = list(np.random.normal(size=(m, n)))
    return matrix

Сумму элементов двумерного массива chunk  по строкам и по элементам каждой строки.

In [71]:
%%file f.py

def sum_by_chunk(chunk):
    matrix_sum = sum(sum(row) for row in chunk)
    return matrix_sum

Overwriting f.py


In [72]:
from f import sum_by_chunk

In [76]:
a = create_2d_list(100000, 1000)

1 ВАР

In [77]:
%%time
sum_by_chunk(a)

Wall time: 6.57 s


16677.273437892036

2 ВАР

In [79]:
def divide(matrix):
    chunks = []
    chunk_size = 1000
    for i in range(0, len(matrix), chunk_size):
        chunks.append(matrix[i:i+chunk_size])
    return chunks
matrix_parts = divide(a)

Возвращает подматрицу, начинающуюся с индекса i и заканчивающуюся индексом i+chunk_size-1. Таким образом, каждая следующая часть матрицы начинается с того индекса, на котором закончилась предыдущая часть

In [80]:
%%time
with Pool(processes=16) as pool:
    print(sum(pool.map(sum_by_chunk, matrix_parts)))

16677.2734378921
Wall time: 4.52 s


3 ВАР

In [81]:
%%time
with ThreadPool(processes=16) as pool:
    print(sum(pool.map(sum_by_chunk, matrix_parts)))

16677.2734378921
Wall time: 6.68 s


каждому потоку будет передана своя подматрица chunk из списка matrix_parts

<p class="task" id="9"></p>

9\. Напишите функцию `create_2d_arr`, которая создает матрицу размера `m` на `n` (__в виде массива numpy__) вещественных чисел из стандартного нормального распределения. Напишите функцию `sum_by_chunk_np`, которая принимает на вход несколько строк этой матрицы (тоже в виде массива) и находит сумму элементов. 

Используя данную функцию, решите задачу поиска суммы по всей матрице тремя способами аналогично задаче 8.

Для демонстрации результата создайте матрицу достаточно большого размера (не менее 100 тыс. строк), выведите на экран результаты работы трех вариантов решения и измерьте время выполнения каждого из них.

В данном задании при поиска суммы не используйте встроенную функцию `sum`, вместо этого используйте возможности `numpy`.

In [84]:
def create_2d_arr(m, n):
    matrix = np.random.normal(size=(m, n))
    return matrix

In [85]:
a= create_2d_arr(100000, 1000)

In [87]:
%%file f.py

def sum_by_chunk(chunk):
    matrix_sum = sum(sum(row) for row in chunk)
    return matrix_sum

Overwriting f.py


1 вар

In [88]:
%%time
sum_by_chunk(a)

Wall time: 6.5 s


13799.437806641754

2 вар

In [89]:
def divide(matrix):
    chunks = []
    chunk_size = 1000
    for i in range(0, len(matrix), chunk_size):
        chunks.append(matrix[i:i+chunk_size])
    return chunks
matrix_parts = divide(a)

In [90]:
%%time
with Pool(processes=16) as pool:
    print(sum(pool.map(sum_by_chunk, matrix_parts)))

13799.437806641674
Wall time: 3.68 s


3 способ

In [91]:
%%time
with ThreadPool(processes=16) as pool:
    print(sum(pool.map(sum_by_chunk, matrix_parts)))

13799.437806641674
Wall time: 6.76 s
